In [1]:
# Import files distance_matrix_1.csv and distance_matrix_2.csv
# Voor foto: ![title](assumptions.png) in markdown cell
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gurobipy import * 
import math 
import time
import seaborn as sns


small_instance = False               # kleine dataset
dataset = 1                         # kies 1 of 2

# Import the data with delimeter as ; in stad of comma

if dataset == 1:
    distances = pd.read_excel('data.xlsx', sheet_name='Distance 1', index_col=0)
    demand = pd.read_excel('data.xlsx', sheet_name='Demand 1')

if dataset == 2:
    distances = pd.read_excel('data.xlsx', sheet_name='Distance 2', index_col=0)
    demand = pd.read_excel('data.xlsx', sheet_name='Demand 2')

elif small_instance:
    distances = pd.read_excel('data.xlsx', sheet_name='Distance_small', index_col=0)
    demand = pd.read_excel('data.xlsx', sheet_name='Demand_small')

nodes = list(distances.columns) # first and last nodes are both the depot (0 and 14, for start and end point of the routes
for i in nodes:
    distances[i][i] = 99999   # penalise distance from node to itself so the model does not use these edges
distances[0][nodes[-1]] = 99999      # penalise distance from depot to itself
distances[nodes[-1]][0] = 99999     # penalise distance from depot to itself

# Single Depot - Remove last row and column of distance data:
distances = distances.iloc[:-1]
distances = distances.iloc[:, :-1]
demand = demand.iloc[:-1]


# After loading the data
distances.columns = distances.index
distances


/var/folders/_q/q2_1h6p13hv3wff2c35rw6zh0000gn/T/ipykernel_2577/633717930.py:31: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  distances[i][i] = 99999   # penalise distance from node to itself so the model does not use these edges
/var/folde

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,99999,55,75,115,170,148,145,159,98,101,95,50,58,20
1,55,99999,43,115,165,155,170,198,142,138,115,75,105,75
2,75,43,99999,80,125,120,142,181,135,123,91,64,106,95
3,115,115,80,99999,56,41,72,124,102,80,40,66,100,124
4,170,165,125,56,99999,41,85,145,145,120,88,121,151,178
5,148,155,120,41,41,99999,45,105,105,80,55,96,120,153
6,145,170,142,72,85,45,99999,60,75,53,53,98,103,145
7,159,198,181,124,145,105,60,99999,63,60,91,123,104,150
8,98,142,135,102,145,105,75,63,99999,25,60,72,40,88
9,101,138,123,80,120,80,53,60,25,99999,40,65,50,95


# Get Vehicle data

In [2]:
vehicles = pd.read_csv('vehicles_10.csv')
vehicles 


,Unnamed: 0,ID,Capacity,Range,Fuel Consumption,Cost
0,0,1,150.0,350.0,1.35,440.0
1,1,2,200.0,450.0,2.00,600.0
2,2,3,200.0,450.0,1.88,600.0
3,3,4,190.0,440.0,1.68,570.0
4,4,5,100.0,260.0,1.02,310.0
5,5,6,100.0,260.0,1.02,310.0
6,6,7,100.0,220.0,1.00,300.0
7,7,8,200.0,450.0,2.00,600.0
8,8,9,190.0,440.0,1.69,570.0
9,9,10,200.0,450.0,1.85,600.0


In [3]:
demand

,Node,Delivery,Pick-up
0,0,0,0
1,1,20,30
2,2,25,30
3,3,15,15
4,4,40,30
5,5,20,15
6,6,10,15
7,7,30,20
8,8,30,35
9,9,25,10


# Sets and Parameters

In [4]:
# differ fleet size and capacity with sensitivity analysis 
n_customer_nodes = distances.shape[0] - 1
p_f = 1.55 # Fuel price


# Create Node and Edge classes 
class Node():

    def __init__(self, nr, pickup_demand, delivery_demand):
        self.nr = nr
        self.pickup_demand = pickup_demand
        self.delivery_demand = delivery_demand

class Edge():

    def __init__(self, origin, destination, length):
        self.origin = origin
        self.destination = destination       
        self.length = length        # (or cost)
        
class Vehicle():

    def __init__(self, id, capacity, range, fuel_consumption, cost):
        self.id = id
        self.Q = capacity              # Capacity
        self.TL = range            # Maximum route length 
        self.f = fuel_consumption
        self.c = cost

# Generate Node and Edge objects from provided benchmark data
        
# Customer nodes N 
N = []
for index, row in demand.iterrows():
    if index in range(1, n_customer_nodes+1):
        node = Node(index, row['Pick-up'], row['Delivery'])
        N.append(node)

# T = [N ∪ {0}], all customer nodes in graph plus depot as start (0), 
T = []
for index, row in demand.iterrows():
    node = Node(index, row['Pick-up'], row['Delivery'])
    T.append(node)

V = []
for index, row in vehicles.iterrows(): 
    vehicle = Vehicle(row['ID'], row['Capacity'], row['Range'], row['Fuel Consumption'], row['Cost'])
    V.append(vehicle)

edges = []
for i in T:
    for j in T:
        origin = i.nr 
        destination = j.nr
        length = distances[i.nr][j.nr]
        edges.append(Edge(origin, destination, length))
 

print(f'\nSet N: \n')
for i in N:
    print(f'Node nr: {i.nr}, Pickup Demand: {i.pickup_demand}, Delivery Demand: {i.delivery_demand}')

print(f'\nSet T: \n')
for i in T:
    print(f'Node nr: {i.nr}, Pickup Demand: {i.pickup_demand}, Delivery Demand: {i.delivery_demand}')

print(f'\nSet V: \n')
for v in V:
    print(f'Vehicle {v.id}: Capacity {v.Q}, Max route length {v.TL}, Fuel Consumption {v.f}, Cost {v.c}')


Set N: 

Node nr: 1, Pickup Demand: 30, Delivery Demand: 20
Node nr: 2, Pickup Demand: 30, Delivery Demand: 25
Node nr: 3, Pickup Demand: 15, Delivery Demand: 15
Node nr: 4, Pickup Demand: 30, Delivery Demand: 40
Node nr: 5, Pickup Demand: 15, Delivery Demand: 20
Node nr: 6, Pickup Demand: 15, Delivery Demand: 10
Node nr: 7, Pickup Demand: 20, Delivery Demand: 30
Node nr: 8, Pickup Demand: 35, Delivery Demand: 30
Node nr: 9, Pickup Demand: 10, Delivery Demand: 25
Node nr: 10, Pickup Demand: 15, Delivery Demand: 20
Node nr: 11, Pickup Demand: 15, Delivery Demand: 15
Node nr: 12, Pickup Demand: 30, Delivery Demand: 20
Node nr: 13, Pickup Demand: 30, Delivery Demand: 20

Set T: 

Node nr: 0, Pickup Demand: 0, Delivery Demand: 0
Node nr: 1, Pickup Demand: 30, Delivery Demand: 20
Node nr: 2, Pickup Demand: 30, Delivery Demand: 25
Node nr: 3, Pickup Demand: 15, Delivery Demand: 15
Node nr: 4, Pickup Demand: 30, Delivery Demand: 40
Node nr: 5, Pickup Demand: 15, Delivery Demand: 20
Node nr: 

# Variables

In [5]:
%%time
m = Model('VRPSPD_var')
##################### Decision Variables ########################
X = {}          # X_ijv = 1 if vehicle v travels from i to j, 0 otherwise
D = {}          # D_iv = The load remaining to be delivered by vehicle v when departing from node i - non-negative integer 
P = {}          # P_iv = The cumulative load picked by vehicle v when departing from node i, - non-negative integer 
y = {}          # y^v = 1 if vehicle v is used, 0 otherwise. 

# X
for i in T:
    for j in T:
        for v in V:
            L_ij = distances[i.nr][j.nr] 
            X[i.nr, j.nr, v.id] = m.addVar(obj = L_ij * v.f * p_f, lb=0,
                                vtype = GRB.BINARY, name = f'X_{i.nr},{j.nr},{v.id}')

# y            
for v in V:
    y[v.id] = m.addVar(obj = v.c, lb=0,
            vtype = GRB.BINARY, name = f'y^{v.id}')

# D & P
for i in T:
    for v in V:
        # D
        D[i.nr, v.id] = m.addVar(obj = 0, lb=0,
                        vtype=GRB.INTEGER, name = f'D_{i.nr},{v.id}')
        
        # P
        P[i.nr, v.id] = m.addVar(obj = 0, lb=0,
                        vtype=GRB.INTEGER, name = f'P_{i.nr},{v.id}')

m.update()
m.setObjective(m.getObjective(), GRB.MINIMIZE)
# m.ModelSense = GRB.MINIMIZE
# m.setObjective( quicksum(quicksum(quicksum( distances[i][j] * v.f * X[i.nr, j.nr, v.id] for i in T) for j in T) for v in V) 
#                 + quicksum(v.c * y[v.id] for v in V) )

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-25
CPU times: user 13.3 ms, sys: 2.53 ms, total: 15.8 ms
Wall time: 16.4 ms


# Constraints

In [6]:
%%time
#################################################### Constraints ###########################################
# Big M:
M = 1000000


#################################################### C1 ####################################################
# Stipulates that each customer node must be visited by exactly once.
for j in N:  
    m.addConstr(quicksum(quicksum( X[i.nr, j.nr, v.id] for i in T) for v in V), 
                GRB.EQUAL, 1, name=f'C1_{j.nr}')

#################################################### C1* ####################################################
# Variant constraint, ensures a vehicle does not visit any node if it is not used. 
for v in V:
     m.addConstr(quicksum(quicksum( X[i.nr, j.nr, v.id] for i in T) for j in T), GRB.LESS_EQUAL,
                                    M * y[v.id], name = f'C1*_{v.id}')


#################################################### C2 ####################################################
#  Ensures that if a vehicle is used, it leaves and enters the depot once. 
for v in V:
    m.addConstr( quicksum(X[0, j.nr, v.id] for j in N), GRB.EQUAL, y[v.id], name=f'C2_{v.id}')
    m.addConstr( quicksum(X[j.nr, 0, v.id] for j in N), GRB.EQUAL, y[v.id], name=f'C2*_{v.id}')



#################################################### C3 ####################################################
# Ensures that a vehicle arrives and departs from each node it serves.
for j in T: 
    for v in V:
        # C3
        m.addConstr(quicksum( X[i.nr, j.nr, v.id] for i in T)  
                    - quicksum( X[j.nr, i.nr, v.id] for i in T), GRB.EQUAL, 0, name=f'C3_{j.nr}_{v.id}') 



#################################################### C4 ###################################################
# C4
for i in T: 
    for v in V:
        # Ensures that the load on vehicle v, when departing from node i, is always lower than the vehicle capacity.
        m.addConstr( D[i.nr, v.id] + P[i.nr, v.id], GRB.LESS_EQUAL, v.Q * y[v.id], name=f'C4_{i.nr}{v.id}')



#################################################### C5 & C6 ###############################################  
# Ensures that the total delivery load for a route is placed on the vehicle v, embarking on each trip, at the starting node itself. 
for v in V: 
    # C5
    m.addConstr(D[0, v.id] - (quicksum(quicksum((X[i.nr, j.nr, v.id] * i.delivery_demand) for i in N) for j in T)), 
                GRB.EQUAL,  0, name=f'C5_{v.id}') 

for v in V:          
    # C6 
    m.addConstr(P[0, v.id], GRB.EQUAL, 0, name='C6')
 


#################################################### C7 & C8 ################################################
# Transit load constraints i.e., if arc (i, j) is visited by the vehicle v, 
# then the quantity to be delivered by the vehicle has to decrease by d_j while 
# the quantity picked-up has to increase by p_j
for v in V:
    for i in T: 
        for j in N: 
            # C7
            m.addConstr( (D[i.nr, v.id] - j.delivery_demand - D[j.nr, v.id]) * X[i.nr, j.nr, v.id], GRB.EQUAL, 0, name=f'C7_{i.nr}_{j.nr}_{v.id}')
for v in V:
    for i in T: 
        for j in N: 
            # C8
            m.addConstr( (P[i.nr, v.id] + j.pickup_demand - P[j.nr, v.id]) * X[i.nr, j.nr, v.id], GRB.EQUAL, 0, name=f'C8_{i.nr}_{j.nr}_{v.id}')


####################################################  C9  #####################################################
# Ensures the maximum route length for any vehicle v. 
for v in V:
    m.addConstr(quicksum(quicksum( distances[i.nr][j.nr] * X[i.nr, j.nr, v.id] for j in T) for i in T), GRB.LESS_EQUAL, v.TL * y[v.id], name=f'C9_{v.id}')



#################################################### C10 & C11 ###############################################
for v in V:
    for i in T:
        # C10
        m.addConstr(D[i.nr, v.id], GRB.GREATER_EQUAL, 0, name='C10')

        # C11
        m.addConstr(P[i.nr, v.id], GRB.GREATER_EQUAL, 0, name='C11')


<timed exec>:9: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
<timed exec>:15: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
<timed exec>:22: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
<timed exec>:23: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
<timed exec>:32: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
<timed exec>:42: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
<timed exec>:50: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
<timed exec>:55: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
<timed exec>:67: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
<timed exec>:72: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
<timed exec>:78: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr


CPU times: user 158 ms, sys: 2.98 ms, total: 161 ms
Wall time: 161 ms


<timed exec>:86: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
<timed exec>:89: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr


# Optimization

In [7]:
m.write('MODEL_var.lp')
# Set time constraint for optimization (5minutes)
m.setParam('TimeLimit', 1200)
# Set gap constraint for optimisation
# m.setParam('MIPgap', 0.05)

m.optimize()
m.write("solution_var.sol")
m.write("testout_var.sol")

status = m.status
if status == GRB.Status.UNBOUNDED:
    print('The model cannot be solved because it is unbounded')

elif status == GRB.Status.OPTIMAL or True:
    f_objective = m.objVal
    print('***** RESULTS ******')
    print('\nObjective Function Value: \t %g' % f_objective)

elif status != GRB.Status.INF_OR_UNBD and status != GRB.Status.INFEASIBLE:
    print('Optimization was stopped with status %d' % status)

Set parameter TimeLimit to value 1200
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 633 rows, 2250 columns and 12220 nonzeros
Model fingerprint: 0xc40eef42
Model has 3640 quadratic constraints
Variable types: 0 continuous, 2250 integer (1970 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+01, 4e+01]
  Objective range  [2e+01, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve added 6070 rows and 0 columns
Presolve removed 0 rows and 256 columns
Presolve time: 0.06s
Presolved: 6703 rows, 1994 columns, 29188 nonzeros
Variable types: 0 continuous, 1994 integer (1724 binary)

Root relaxation: objective 1.394366e+03, 1023 iterations, 0.03 seconds (0.06 work units)

    Nodes    |    Current Node    |     Objective Bounds      |  

# Verification

In [8]:
# Test function for checking feasibility:
def get_routes(V, T):
    route_lengths = {}
    routes = {}
    for v in V:
        if y[v.id].X > 0.1:
            # print(f'\nVehicle {v.id}:')
            routes[v.id] = []
            route_length = 0
            for i in T:
                for j in T:
                    if X[i.nr, j.nr, v.id].X > 0.01:
                        # print(f'  {i.nr} --> {j.nr}') 
                        routes[v.id].append((i.nr, j.nr))
                        route_length += distances[i.nr][j.nr]
            route_lengths[v.id] = route_length

    return routes, route_lengths

def sort_routes(route_dict):
    """
    Sorts every route in the right order. Routes start and end at 0.
    
    :param route_dict: Dictionary containing routes with route IDs as keys
                       and lists of tuples (start, end) as values.
    :return: Dictionary with sorted routes.
    """
    sorted_routes = {}
    for vehicle, route in route_dict.items():
        # Start with an empty route
        sorted_route = []
        # Current location starts at 0 (depot)
        current_location = 0
        
        while route:
            # Find the next step in the route where the first element is the current location
            for i, (start, end) in enumerate(route):
                if start == current_location:
                    # Add this step to the sorted route
                    sorted_route.append((start, end))
                    # Remove this step from the route
                    route.pop(i)
                    # Update the current location to the end of this step
                    current_location = end
                    break
                    
        sorted_routes[vehicle] = sorted_route

    # # print routes
    # for vehicle, route in sorted_routes.items():
    #     # print(f'\nVehicle {vehicle}:')
    #     for (i, j) in route:
    #         if X[i, j, vehicle].X > 0.01:
    #             # print(f'  {i} --> {j}') 
    return sorted_routes

def print_routes(routes):
    for vehicle, route in routes.items():
        print(f'\nVehicle {vehicle}:')
        for (i, j) in route:
            if X[i, j, vehicle].X > 0.01:
                print(f'  {i} --> {j}') 

def double_visit(nodes, node):
    count = 0
    for number in nodes:
        if number == node:
            count += 1
    return True if count > 1 else False

def double_depart(nodes, node):
    count = 0
    for number in nodes:
        if number == node:
            count += 1
    return True if count > 1 else False

def customer_node_double_visit(routes, node):
    count = 0
    for vehicle, route in routes.items():
        for edge in route:
            visited = edge[1]
            if node == visited:
                count += 1
    return True if count > 1 else False

def delivery_and_pickup_demand(vehicle_id, route):
    '''returns the required total amounts to be deliverd and picked up in each route'''
    required_D = 0
    required_P = 0
    for edge in route:
        i = edge[0]
        j = edge[1]
        # print(f'From {i} to {j}')
        # print(f'Delivered = {demand["Delivery"].iloc[i]}')
        # print(f'Picked up = {demand["Pick-up"].iloc[i]}')
        required_D += demand["Delivery"].iloc[i]
        required_P += demand["Pick-up"].iloc[i]
    #     print(f' D_{i}, P_{i} = {required_D}, {required_P}')
    # print(f' D and P: {required_D}, {required_P}')    
    return required_D, required_P

def max_vehicle_load(vehicle_id, route):
    '''returns the maximum load a vehicle has carried in a certain route'''
    max_load = 0
    for edge in route:
        i = edge[0]
        j = edge[1]
        load = D[i, vehicle_id].X + P[i, vehicle_id].X
        j_pickup_demand = demand["Pick-up"].iloc[j]
        # print(f' \nEdge {edge}')
        # print(f' D_{i},{vehicle_id} = {D[i, vehicle_id].X}')
        # print(f' P_{i},{vehicle_id} = {P[i, vehicle_id].X}')
        # print(f'\nC8:')
        # print(f'(P_{i},{vehicle_id} + {j_pickup_demand} - P_{j},{vehicle_id}) * X_{i},{j},{vehicle_id} = 0')
        # print(f'({P[i, vehicle_id].X} + {j_pickup_demand} - {P[j, vehicle_id].X}) * {X[i, j, vehicle_id].X} = 0')
        if load > max_load:
            max_load = load

    return max_load 

def check_subtours_and_double_visits(routes):
    '''Check routes for subtours and double visits, return True or False for feasibility'''
    for vehicle, route in routes.items():
        nodes_to = [edge[1] for edge in route]
        nodes_from = [edge[0] for edge in route]
        # print(f'nodes_to {nodes_to} v={vehicle}')
        # print(f'nodes_from: {nodes_from} v={vehicle}')

        # Check if nodes are departed from more than once by same vehicle
        for node in nodes_from:
            if double_depart(nodes_from, node): 
                print(f'Route Infeasible:\n Node {node} departed from more than once by vehicle {vehicle}.')
                return False

        # Check if nodes are visited more than once by same vehicle 
        for node in nodes_to:
            if double_visit(nodes_to, node):     
                print(f'Route Infeasible:\n Node {node} visited more than once by Vehicle {vehicle}.')
                return False

        # Check if any customer nodes are visited more than once
        for customer_node in N:
            if customer_node_double_visit(routes, customer_node.nr):
                print(f'Route Infeasible:\n Customer node {customer_node.nr} visited more than once.')
                return False
    return True


# Results

In [9]:
# Get results:
print(f'Objective Function value: {f_objective}')
routes, route_lengths = get_routes(V, T)

# Check for subtours and double visits:
if check_subtours_and_double_visits(routes) == True:
    # sort routes if the routes are contain no subtours or double visits of customer nodes
    routes = sort_routes(routes) 

# print routes:    
print_routes(routes)
print(f'Route lengths: {route_lengths}')


# Check if the maximum loads and route lengths do not exceed capacity
for vehicle, route in routes.items(): 
    v = next((v for v in V if v.id == vehicle), None)
    
    max_load = max_vehicle_load(vehicle, route)
    distance_travelled = route_lengths[vehicle]
    D_required, P_required = delivery_and_pickup_demand(vehicle, route)
    last_customer_node = route[-1][0]
    D_delivered, P_pickedup = D[0, vehicle].X, P[last_customer_node, vehicle].X
    


    # Test delivery and pickup quantities
    if abs(D_required - D_delivered) > 0.01:
        print(f'INFEASIBLE:\nDelivered quantities ({D_delivered}) by vehicle {vehicle} do not match demand ({D_required})')
        break
    
    if abs(P_required - P_pickedup) > 0.01:
        print(f'INFEASIBLE:\nPicked up quantities ({P_pickedup}) by vehicle {vehicle} do not match demand ({P_required})')
        break

    # Test max load
    if max_load > v.Q:
        print(f'INFEASIBLE:\nVehicle {v.id} exceeded capacity')
        break

    # Test max distance
    if distance_travelled > v.TL:
        print(f'INFEASIBLE:\n\n Vehicle {v.id} exceeded Max Route Length')
        break

    print('\n')
    print(f'Vehicle {vehicle}, Capacity {v.Q}, Max route length {v.TL}:')
    print(f'    Max load carried by Vehicle {vehicle} is {max_load}')
    print(f'    Distance travelled by Vehicle {vehicle} is {route_lengths[vehicle]}')
    print(f'    Loads deliverd and picked-up: {D_delivered}, {P_pickedup}')
    print(f'    Demand for delivery and pickup: {D_required}, {P_required}')

    

Objective Function value: 2162.2200000000003

Vehicle 4.0:
  0 --> 1
  1 --> 2
  2 --> 3
  3 --> 4
  4 --> 5
  5 --> 10
  10 --> 11
  11 --> 0

Vehicle 9.0:
  0 --> 9
  9 --> 6
  6 --> 7
  7 --> 8
  8 --> 12
  12 --> 13
  13 --> 0
Route lengths: {4.0: 425, 9.0: 384}


Vehicle 4.0, Capacity 190.0, Max route length 440.0:
    Max load carried by Vehicle 4.0 is 170.0
    Distance travelled by Vehicle 4.0 is 425
    Loads deliverd and picked-up: 155.0, 150.0
    Demand for delivery and pickup: 155, 150


Vehicle 9.0, Capacity 190.0, Max route length 440.0:
    Max load carried by Vehicle 9.0 is 140.0
    Distance travelled by Vehicle 9.0 is 384
    Loads deliverd and picked-up: 135.0, 140.0
    Demand for delivery and pickup: 135, 140


In [10]:
## Sensitivity Analysis

# Sensitivity analysis for fuel price

# def sensitivity_analysis_fuel_price(p_f):
#     m.setObjective(m.getObjective(), GRB.MINIMIZE)
#     m.setObjective( quicksum(quicksum(quicksum( distances[i.nr][j.nr] * v.f * X[i.nr, j.nr, v.id] for i in T) for j in T) for v in V) 
#                     + quicksum(v.c * y[v.id] for v in V) )
#     m.optimize()
#     f_objective = m.objVal
#     print(f'\nObjective Function value for fuel price {p_f}: {f_objective}')
#     routes, route_lengths = get_routes(V, T)
#     if check_subtours_and_double_visits(routes) == True:
#         routes = sort_routes(routes) 
#     print_routes(routes)
#     print(f'Route lengths: {route_lengths}')
#     return f_objective

# # Sensitivity analysis for fuel price
# fuel_prices = [0.5, 0.75, 1.0, 1.25, 1.5]
# results = {}
# for p_f in fuel_prices:
#     results[p_f] = sensitivity_analysis_fuel_price(p_f)

# # Plot results
# plt.plot(list(results.keys()), list(results.values()))
# plt.xlabel('Fuel Price')
# plt.ylabel('Objective Function Value')
# plt.title('Sensitivity Analysis for Fuel Price')
# plt.show()